In [1]:
import pandas as pd
import pandas

In [2]:
%matplotlib inline
from numpy import array, linspace ; from numpy.random import randint
from matplotlib.pyplot import hist, xticks, show
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
# synthesize some data 
font = {
    'family' : 'serif',
    'weight':'normal',
        'size'   : 12}

matplotlib.rc('font', **font)
import seaborn as sns
pallete = sns.color_palette("Set1", 4)
dgl_color = pallete[0]
quiver_color = pallete[1]
groot_color = pallete[2]
p3_color = pallete[3]
hatches = list('-*/o')

pallete = sns.color_palette("Set1", 5)
pallete

[(0.8941176470588236, 0.10196078431372549, 0.10980392156862745),
 (0.21568627450980393, 0.49411764705882355, 0.7215686274509804),
 (0.30196078431372547, 0.6862745098039216, 0.2901960784313726),
 (0.596078431372549, 0.3058823529411765, 0.6392156862745098),
 (1.0, 0.4980392156862745, 0.0)]

In [3]:
import pandas as pd
machine_name = 'p3.8xlarge/nvlink'
# machine_name = 'g4dn.12xlarge/pcie'
df1 = pd.read_csv(f'march_logs/{machine_name}_default.csv')
df2 = pd.read_csv(f'march_logs/{machine_name}_partition.csv')
df = pd.concat([df1,df2], axis = 0)

print(df.columns)
# print(df)
d = {'system': 1, 'b': 2, 'c': 3}
dfs = []
idx = ['model','system']
dgl_cache_pers = [1, 0, 0, 0,]
graphs = ['ogbn-papers100M']
graphs = ['ogbn-papers100M','com-orkut','com-friendster']
graphs = ['papers100M','orkut','friendster']
# graphs = ['friendster']
# graphs = []
# , 'com-orkut', 'com-friendster']
for graph_name in graphs:
    idx.append(f'sampling_{graph_name}')
    idx.append(f'loading_{graph_name}')
    idx.append(f'training_{graph_name}')
    idx.append(f'total_{graph_name}')
batch_size = 1024
system_label = {}
system_label = {'ndst_efreq_xbal':'score+','ndegree_euniform_bal':'edge-bal'}
graph_label = {'ogbn-products':"Products", "ogbn-papers100M":"Papers100M", \
                   "com-orkut":"Orkut", "com-friendster":"Friendster"}
graph_label = {'ogbn-products':"Products", "papers100M":"Papers100M", \
                   "orkut":"Orkut", "friendster":"Friendster"}
dgl_time = {}
for dataset_id,graph_name in enumerate(graphs):
#     for system in ['dgl','p3','quiver','groot']:
    for partitioning in ['ndegree_euniform_bal', 'ndst_efreq_xbal']:
#         print(dataset_id, graph_name, system)
        out = {}
        out['graph_name'] = graph_label[graph_name]
        out['partitioning'] = system_label[partitioning]
        for model in ["sage", "gat"]:
            dgl_cache_per = dgl_cache_pers[dataset_id]
            dataset_df = df[(df['graph_name'] == graph_name) & (df['model'] == model)\
                        & (df['batch_size'] == batch_size) & \
                            (df['system'].str.contains('split') & (df['partition_type'] == partitioning))\
                           ]
            row = dataset_df
            if len(row) != 1:
                print(row)
            sampling_time = row['sampling (s)'].item()
            if sampling_time != "oom":
                n_epochs = float(row['num_epoch'].item())
                sampling_time = float(sampling_time)/n_epochs
                data_loading = float(row['feature (s)'].item())/n_epochs
                training_time = (float(row['forward (s)'].item()) + float(row['backward (s)'].item()))/n_epochs
                out[f'sampling_{model}'] = "{:.2f}".format(sampling_time)
                out[f'loading_{model}'] = "{:.2f}".format(data_loading)
                out[f'data_{model}'] = "{:.2f}".format(training_time)
                training_time = sampling_time + data_loading + training_time
                out[f'total_{model}'] = "{:.2f}".format(training_time)
                
            else:
                out[f'sampling_{model}'] = "OOM"
                out[f'loading_{model}'] = "OOM"
                out[f'data_{model}'] = "OOM"
                out[f'total_{model}'] = "OOM"
                  
        dfs.append(pd.Series(out).to_frame().T)
final = (pd.concat(dfs, ignore_index = True))
print(final.to_csv(index = False))
# print(final)
print(machine_name)

Index(['timestamp', 'machine_name', 'graph_name', 'feat_width', 'world_size',
       'num_partition', 'num_epoch', 'fanouts', 'num_redundant_layers',
       'batch_size', 'system', 'model', 'hid_size', 'cache_size',
       'partition_type', 'duration (s)', 'sampling (s)', 'feature (s)',
       'forward (s)', 'backward (s)', 'allocated (MB)', 'reserved (MB)',
       'test accuracy %', 'edges_computed', 'edge_skew', 'min_edge',
       'max_edge', 'run_time'],
      dtype='object')
graph_name,partitioning,sampling_sage,loading_sage,data_sage,total_sage,sampling_gat,loading_gat,data_gat,total_gat
Papers100M,edge-bal,6.08,0.06,8.01,14.14,6.04,0.06,19.93,26.03
Papers100M,score+,2.17,1.44,6.29,9.90,2.34,1.25,16.34,19.93
Orkut,edge-bal,2.54,0.67,12.48,15.70,2.53,0.67,16.68,19.87
Orkut,score+,2.16,0.04,7.31,9.52,2.15,0.04,10.35,12.55
Friendster,edge-bal,50.42,0.71,59.41,110.55,57.87,0.55,175.21,233.62
Friendster,score+,36.24,2.52,50.24,89.00,36.08,2.48,142.75,181.30

p3.8xlarge/nvlink
